In [18]:
import pandas as pd
import os

def main():
    # Пути к файлам
    data_dir = r"C:\Users\rusla\OneDrive\Рабочий стол\ctd2"
    depmap_dir = r"C:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\depmap_datasets\prepared_datasets"
    
    curves_path = os.path.join(data_dir, "v20.data.curves_post_qc.txt")
    meta_exp_path = os.path.join(data_dir, "v20.meta.per_experiment.txt")
    meta_cpd_path = os.path.join(data_dir, "v20.meta.per_compound.txt")
    meta_ccl_path = os.path.join(data_dir, "v20.meta.per_cell_line.txt")
    depmap_path = os.path.join(depmap_dir, "cancer_cell_lines_merged_data.csv")
    
    # Чтение всех файлов
    print("Чтение файлов...")
    curves_df = pd.read_csv(curves_path, sep='\t')
    meta_exp_df = pd.read_csv(meta_exp_path, sep='\t')
    meta_cpd_df = pd.read_csv(meta_cpd_path, sep='\t')
    meta_ccl_df = pd.read_csv(meta_ccl_path, sep='\t')
    depmap_df = pd.read_csv(depmap_path)
    
    # 1. Выбор нужных столбцов из данных кривых
    curves_df = curves_df[['experiment_id', 'area_under_curve', 'master_cpd_id']]
    
    # 2. Добавление master_ccl_id из экспериментальных метаданных
    meta_exp_df = meta_exp_df[['experiment_id', 'master_ccl_id']]
    merged_df = pd.merge(curves_df, meta_exp_df, on='experiment_id', how='left')
    
    # 3. Добавление информации о соединениях
    cpd_info_df = meta_cpd_df[['master_cpd_id', 'cpd_name', 'broad_cpd_id']]
    merged_df = pd.merge(merged_df, cpd_info_df, on='master_cpd_id', how='left')
    
    # 4. Добавление информации о клеточных линиях
    ccl_info_df = meta_ccl_df[['master_ccl_id', 'ccl_name']]
    merged_df = pd.merge(merged_df, ccl_info_df, on='master_ccl_id', how='left')
    
    # 5. Добавление информации из DepMap
    depmap_info_df = depmap_df[['StrippedCellLineName', 'ModelID', 'primary_disease_renamed']]
    merged_df = pd.merge(
        merged_df, 
        depmap_info_df, 
        left_on='ccl_name', 
        right_on='StrippedCellLineName', 
        how='inner'
    ).drop('StrippedCellLineName', axis=1)
    
    # 6. Удаление дубликатов (оставляем строку с минимальным area_under_curve)
    print("Удаление дубликатов...")
    merged_df = merged_df.sort_values('area_under_curve').drop_duplicates(
        subset=['ModelID', 'master_cpd_id'], 
        keep='first'
    )
    
    # Определение пути сохранения
    output_filename = "final_filtered_dedup_merged_result.csv"
    try:
        script_dir = os.path.dirname(os.path.abspath(__file__))
    except NameError:
        script_dir = os.getcwd()
    output_path = os.path.join(script_dir, output_filename)
    
    # Сохранение результата
    print(f"Сохранение результата в {output_path}...")
    merged_df.to_csv(output_path, index=False)
    
    # Вывод информации
    print("\n✅ Файл успешно сохранен!")
    print(f"Сохранено строк: {len(merged_df)}")
    print("\nСтруктура итогового файла:")
    print(merged_df.info())
    
    print("\nПервые 5 строк результата:")
    print(merged_df.head())
    
    print("\nСтатистика по данным:")
    print(f"Уникальных комбинаций ModelID + master_cpd_id: {len(merged_df)}")
    print(f"Уникальных experiment_id: {merged_df['experiment_id'].nunique()}")
    print(f"Уникальных соединений: {merged_df['master_cpd_id'].nunique()}")
    print(f"Уникальных клеточных линий: {merged_df['ModelID'].nunique()}")
    print(f"Уникальных заболеваний: {merged_df['primary_disease_renamed'].nunique()}")

if __name__ == "__main__":
    main()

Чтение файлов...
Удаление дубликатов...
Сохранение результата в c:\Users\rusla\OneDrive\Рабочий стол\ctd2\final_filtered_dedup_merged_result.csv...

✅ Файл успешно сохранен!
Сохранено строк: 365321

Структура итогового файла:
<class 'pandas.core.frame.DataFrame'>
Index: 365321 entries, 258412 to 135262
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   experiment_id            365321 non-null  int64  
 1   area_under_curve         365321 non-null  float64
 2   master_cpd_id            365321 non-null  int64  
 3   master_ccl_id            365321 non-null  int64  
 4   cpd_name                 365321 non-null  object 
 5   broad_cpd_id             365321 non-null  object 
 6   ccl_name                 365321 non-null  object 
 7   ModelID                  365321 non-null  object 
 8   primary_disease_renamed  365321 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 27.9+

In [6]:
import pandas as pd
from scipy.stats import zscore
import os

# Загрузка данных (замените на ваш реальный путь к файлу)
data = pd.read_csv(r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\final_filtered_dedup_merged_result.csv")  # Укажите правильный разделитель

# 1. Создание таблицы z-score для препаратов и клеточных линий
# Группируем по препарату и клеточной линии, берем среднее AUC для каждой комбинации
grouped = data.groupby(['master_cpd_id', 'ModelID'])['area_under_curve'].mean().reset_index()

# Вычисляем z-score для каждого препарата среди всех клеточных линий
grouped['z_score'] = grouped.groupby('master_cpd_id')['area_under_curve'].transform(
    lambda x: zscore(x, ddof=1) if len(x) > 1 else pd.NA
)

# Создаем сводную таблицу
pivot_cells = grouped.pivot(index='master_cpd_id', columns='ModelID', values='z_score')

# 2. Создание таблицы z-score для препаратов и типов рака
# Сначала добавляем тип рака к данным
merged = pd.merge(grouped, data[['ModelID', 'primary_disease_renamed']].drop_duplicates(), 
                 on='ModelID', how='left')

# Группируем по препарату и типу рака, берем среднее z-score
disease_avg = merged.groupby(['master_cpd_id', 'primary_disease_renamed'])['z_score'].mean().reset_index()

# Создаем сводную таблицу
pivot_diseases = disease_avg.pivot(index='master_cpd_id', columns='primary_disease_renamed', values='z_score')

# Создаем папку для результатов, если ее нет
os.makedirs('results', exist_ok=True)

# Сохраняем результаты
pivot_cells.to_csv('results/drug_cell_lines_zscore.csv', float_format='%.4f')
pivot_diseases.to_csv('results/drug_diseases_avg_zscore.csv', float_format='%.4f')

print("Файлы успешно сохранены в папку 'results':")
print("- drug_cell_lines_zscore.csv")
print("- drug_diseases_avg_zscore.csv")

Файлы успешно сохранены в папку 'results':
- drug_cell_lines_zscore.csv
- drug_diseases_avg_zscore.csv


In [7]:
import pandas as pd

# Загрузка данных
compounds_path = r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\PortalCompounds.csv"
all_genes_path = r"C:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\thresholds\gene_selection\ALL\ALL_depscore_-0.15_zscore_-0.5.csv"
aml_genes_path = r"C:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\thresholds\gene_selection\AML\AML_depscore_-0.15_zscore_-0.5.csv"

# Чтение файлов
compounds_df = pd.read_csv(compounds_path)
all_genes = pd.read_csv(all_genes_path)['Gene'].tolist()
aml_genes = pd.read_csv(aml_genes_path)['Gene'].tolist()

def filter_compounds(compounds_df, target_genes):
    """Фильтрует препараты по генам-мишеням и оставляет только совпавшие гены"""
    results = []
    
    for _, row in compounds_df.iterrows():
        if pd.isna(row['GeneSymbolOfTargets']):
            continue
            
        # Разделяем гены-мишени
        target_list = [gene.strip() for gene in row['GeneSymbolOfTargets'].split(';')]
        
        # Находим пересечение с целевыми генами
        matched_genes = [gene for gene in target_list if gene in target_genes]
        
        if matched_genes:
            # Создаем новую строку только с совпавшими генами
            new_row = row.copy()
            new_row['GeneSymbolOfTargets'] = ';'.join(matched_genes)
            results.append(new_row)
    
    return pd.DataFrame(results)

# Фильтрация для ALL
all_filtered = filter_compounds(compounds_df, all_genes)
all_filtered.to_csv('filtered_compounds_ALL.csv', index=False)

# Фильтрация для AML
aml_filtered = filter_compounds(compounds_df, aml_genes)
aml_filtered.to_csv('filtered_compounds_AML.csv', index=False)

print(f"Сохранено {len(all_filtered)} препаратов для ALL в filtered_compounds_ALL.csv")
print(f"Сохранено {len(aml_filtered)} препаратов для AML в filtered_compounds_AML.csv")

Сохранено 536 препаратов для ALL в filtered_compounds_ALL.csv
Сохранено 261 препаратов для AML в filtered_compounds_AML.csv


In [26]:
import pandas as pd

# Чтение файлов
compounds_all = pd.read_csv(r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\filtered_compounds_ALL.csv")
compounds_aml = pd.read_csv(r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\filtered_compounds_AML.csv")
id_mapping = pd.read_csv(r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\final_filtered_dedup_merged_result.csv")
z_scores = pd.read_csv(r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\drug_diseases_avg_zscore.csv")

# Проверяем наличие столбцов
print("Столбцы в filtered_compounds_ALL.csv:", compounds_all.columns.tolist())
print("Столбцы в filtered_compounds_AML.csv:", compounds_aml.columns.tolist())
print("Столбцы в final_filtered_dedup_merged_result.csv:", id_mapping.columns.tolist())
print("Столбцы в drug_diseases_avg_zscore.csv:", z_scores.columns.tolist())

# Оставляем только нужные столбцы в z_scores
z_scores = z_scores[['master_cpd_id', 'ALL', 'AML']]

# Функция для извлечения BRD-K... из SampleIDs
def extract_broad_id(sample):
    if not sample or pd.isna(sample):
        return None
    sample = sample.strip()
    if sample.startswith('BRD:'):
        sample = sample[4:]  # Удаляем BRD:
    # Если нет BRD-K, игнорируем (например, CTRP:616354, GDSC1:1242)
    if not sample.startswith('BRD-K'):
        return None
    # Берем только BRD-K... до второго дефиса
    parts = sample.split('-')
    if len(parts) >= 2:
        return f"{parts[0]}-{parts[1]}"  # Например, BRD-K62353271
    return None

# Функция для поиска broad_cpd_id в SampleIDs
def find_broad_id(sample_ids, broad_ids):
    if pd.isna(sample_ids):
        return None
    sample_list = str(sample_ids).split(';')
    for sample in sample_list:
        extracted_id = extract_broad_id(sample)
        if extracted_id and extracted_id in broad_ids:
            return extracted_id
    return None

# Уникальные broad_cpd_id из id_mapping
id_mapping = id_mapping.drop_duplicates(subset='broad_cpd_id', keep='first')
broad_id_set = set(id_mapping['broad_cpd_id'].astype(str).str.strip())

# Проверяем пересечение SampleIDs и broad_cpd_id
all_sample_ids = set()
for sample in compounds_all['SampleIDs'].dropna():
    for s in str(sample).split(';'):
        extracted_id = extract_broad_id(s)
        if extracted_id:
            all_sample_ids.add(extracted_id)
for sample in compounds_aml['SampleIDs'].dropna():
    for s in str(sample).split(';'):
        extracted_id = extract_broad_id(s)
        if extracted_id:
            all_sample_ids.add(extracted_id)

common_ids = all_sample_ids.intersection(broad_id_set)
print(f"Количество общих broad_cpd_id: {len(common_ids)}")
if common_ids:
    print("Примеры общих broad_cpd_id:", list(common_ids)[:5])
else:
    print("Совпадений broad_cpd_id не найдено!")
    print("Примеры обработанных SampleIDs:", list(all_sample_ids)[:5])
    print("Примеры broad_cpd_id:", list(broad_id_set)[:5])

# Создаем выходные датафреймы
output_all = z_scores[['master_cpd_id', 'ALL']].copy()
output_aml = z_scores[['master_cpd_id', 'AML']].copy()

# Добавляем столбцы
for df in [output_all, output_aml]:
    df['CompoundName'] = None
    df['GeneSymbolOfTargets'] = None
    df['TargetOrMechanism'] = None

# Функция для обработки одного набора данных
def process_compounds(compounds_df, output_df, disease):
    compounds_df['found_broad_id'] = compounds_df['SampleIDs'].apply(
        lambda x: find_broad_id(x, broad_id_set)
    )
    
    # Логируем совпадения
    matched_count = compounds_df['found_broad_id'].notna().sum()
    print(f"Найдено совпадений broad_cpd_id для {disease}: {matched_count}")
    if matched_count > 0:
        print("Примеры найденных broad_cpd_id:", compounds_df['found_broad_id'].dropna().head().tolist())
    
    # Фильтруем строки с найденными broad_cpd_id
    matched = compounds_df[compounds_df['found_broad_id'].notna()]
    
    # Соединяем с id_mapping
    matched = matched.merge(
        id_mapping[['broad_cpd_id', 'master_cpd_id']],
        left_on='found_broad_id',
        right_on='broad_cpd_id',
        how='left'
    )
    
    # Логируем master_cpd_id
    master_count = matched['master_cpd_id'].notna().sum()
    print(f"Найдено master_cpd_id для {disease}: {master_count}")
    
    # Обновляем выходной датафрейм
    for _, row in matched.iterrows():
        mask = output_df['master_cpd_id'] == row['master_cpd_id']
        if mask.any():
            output_df.loc[mask, 'CompoundName'] = row.get('CompoundName')
            output_df.loc[mask, 'GeneSymbolOfTargets'] = row.get('GeneSymbolOfTargets')
            output_df.loc[mask, 'TargetOrMechanism'] = row.get('TargetOrMechanism')

# Обрабатываем данные
process_compounds(compounds_all, output_all, 'ALL')
process_compounds(compounds_aml, output_aml, 'AML')

# Сохраняем результаты
output_all.to_csv(r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\output_ALL.csv", index=False)
output_aml.to_csv(r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\output_AML.csv", index=False)

print("Обработка завершена. Файлы сохранены как output_ALL.csv и output_AML.csv")

Столбцы в filtered_compounds_ALL.csv: ['CompoundID', 'CompoundName', 'GeneSymbolOfTargets', 'TargetOrMechanism', 'Synonyms', 'SampleIDs', 'ChEMBLID', 'SMILES', 'InChIKey', 'DoseUnit', 'PubChemCID']
Столбцы в filtered_compounds_AML.csv: ['CompoundID', 'CompoundName', 'GeneSymbolOfTargets', 'TargetOrMechanism', 'Synonyms', 'SampleIDs', 'ChEMBLID', 'SMILES', 'InChIKey', 'DoseUnit', 'PubChemCID']
Столбцы в final_filtered_dedup_merged_result.csv: ['experiment_id', 'area_under_curve', 'master_cpd_id', 'master_ccl_id', 'cpd_name', 'broad_cpd_id', 'ccl_name', 'ModelID', 'primary_disease_renamed']
Столбцы в drug_diseases_avg_zscore.csv: ['master_cpd_id', 'ALL', 'AML', 'Bile Duct Cancer', 'Bladder Cancer', 'Bone Cancer', 'Breast Cancer', 'CLL', 'CML', 'Cervical Cancer', 'Colon/Colorectal Cancer', 'Endometrial/Uterine Cancer', 'Esophageal Cancer', 'Ewing Sarcoma', 'Gallbladder Cancer', 'Gastric Cancer', 'Glioma', 'Head and Neck Cancer', 'Kidney Cancer', 'Liver Cancer', 'Lung Cancer', 'Lymphoma', 

In [27]:
import pandas as pd

# Чтение файлов
output_all = pd.read_csv(r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\output_ALL.csv")
output_aml = pd.read_csv(r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\output_AML.csv")
zscore_all = pd.read_csv(r"C:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\r_visualizations\scatter_plots\mean_gene_z-score_depscore_by_cancer_type\mean_gene_z-score_depscore_data\mean_gene_z-score_depscore_all.csv")
zscore_aml = pd.read_csv(r"C:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\r_visualizations\scatter_plots\mean_gene_z-score_depscore_by_cancer_type\mean_gene_z-score_depscore_data\mean_gene_z-score_depscore_aml.csv")

# Проверяем столбцы
print("Столбцы в output_ALL.csv:", output_all.columns.tolist())
print("Столбцы в output_AML.csv:", output_aml.columns.tolist())
print("Столбцы в mean_gene_z-score_depscore_all.csv:", zscore_all.columns.tolist())
print("Столбцы в mean_gene_z-score_depscore_aml.csv:", zscore_aml.columns.tolist())

# Создаем словари для быстрого поиска Mean_Zscore по гену
zscore_all_dict = dict(zip(zscore_all['Gene'], zscore_all['Mean_Zscore']))
zscore_aml_dict = dict(zip(zscore_aml['Gene'], zscore_aml['Mean_Zscore']))

# Функция для обработки GeneSymbolOfTargets
def update_gene_zscore(gene_str, zscore_dict):
    if pd.isna(gene_str) or not gene_str:
        return gene_str
    genes = gene_str.split(';')
    updated_genes = []
    for gene in genes:
        gene = gene.strip()
        if gene in zscore_dict:
            zscore = zscore_dict[gene]
            updated_genes.append(f"{gene} = {zscore}")
        else:
            updated_genes.append(f"{gene} = Not found")
    return ';'.join(updated_genes)

# Обновляем GeneSymbolOfTargets в output_all
output_all['GeneSymbolOfTargets'] = output_all['GeneSymbolOfTargets'].apply(
    lambda x: update_gene_zscore(x, zscore_all_dict)
)

# Обновляем GeneSymbolOfTargets в output_aml
output_aml['GeneSymbolOfTargets'] = output_aml['GeneSymbolOfTargets'].apply(
    lambda x: update_gene_zscore(x, zscore_aml_dict)
)

# Сохраняем обновленные файлы
output_all.to_csv(r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\output_ALL.csv", index=False)
output_aml.to_csv(r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\output_AML.csv", index=False)

print("Обработка завершена. Файлы output_ALL.csv и output_AML.csv обновлены.")

Столбцы в output_ALL.csv: ['master_cpd_id', 'ALL', 'CompoundName', 'GeneSymbolOfTargets', 'TargetOrMechanism']
Столбцы в output_AML.csv: ['master_cpd_id', 'AML', 'CompoundName', 'GeneSymbolOfTargets', 'TargetOrMechanism']
Столбцы в mean_gene_z-score_depscore_all.csv: ['Gene', 'Mean_DepScore', 'Mean_Zscore']
Столбцы в mean_gene_z-score_depscore_aml.csv: ['Gene', 'Mean_DepScore', 'Mean_Zscore']
Обработка завершена. Файлы output_ALL.csv и output_AML.csv обновлены.


In [28]:
import pandas as pd

# Чтение файлов
id_mapping = pd.read_csv(r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\final_filtered_dedup_merged_result.csv")
output_all = pd.read_csv(r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\output_ALL.csv")
output_aml = pd.read_csv(r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\output_AML.csv")

# Проверяем столбцы
print("Столбцы в final_filtered_dedup_merged_result.csv:", id_mapping.columns.tolist())
print("Столбцы в output_ALL.csv:", output_all.columns.tolist())
print("Столбцы в output_AML.csv:", output_aml.columns.tolist())

# Создаем словарь master_cpd_id -> cpd_name
# Удаляем дубликаты, оставляя первое вхождение
id_mapping = id_mapping[['master_cpd_id', 'cpd_name']].drop_duplicates(subset='master_cpd_id', keep='first')
cpd_name_dict = dict(zip(id_mapping['master_cpd_id'], id_mapping['cpd_name']))

# Логируем размер словаря
print(f"Размер словаря cpd_name: {len(cpd_name_dict)}")
print("Примеры из словаря:", list(cpd_name_dict.items())[:5])

# Функция для добавления столбца cpd_name после master_cpd_id
def add_cpd_name_column(df, cpd_dict):
    # Создаем новый столбец cpd_name, используя словарь
    df['cpd_name'] = df['master_cpd_id'].map(cpd_dict)
    # Получаем список столбцов
    columns = df.columns.tolist()
    # Перемещаем cpd_name после master_cpd_id
    columns.insert(columns.index('master_cpd_id') + 1, columns.pop(columns.index('cpd_name')))
    # Переупорядочиваем столбцы
    return df[columns]

# Обновляем output_all
output_all = add_cpd_name_column(output_all, cpd_name_dict)

# Обновляем output_aml
output_aml = add_cpd_name_column(output_aml, cpd_name_dict)

# Логируем количество заполненных cpd_name
print(f"Заполнено cpd_name в output_ALL.csv: {output_all['cpd_name'].notna().sum()}")
print(f"Заполнено cpd_name в output_AML.csv: {output_aml['cpd_name'].notna().sum()}")

# Сохраняем обновленные файлы
output_all.to_csv(r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\output_ALL.csv", index=False)
output_aml.to_csv(r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\output_AML.csv", index=False)

print("Обработка завершена. Файлы output_ALL.csv и output_AML.csv обновлены.")

Столбцы в final_filtered_dedup_merged_result.csv: ['experiment_id', 'area_under_curve', 'master_cpd_id', 'master_ccl_id', 'cpd_name', 'broad_cpd_id', 'ccl_name', 'ModelID', 'primary_disease_renamed']
Столбцы в output_ALL.csv: ['master_cpd_id', 'ALL', 'CompoundName', 'GeneSymbolOfTargets', 'TargetOrMechanism']
Столбцы в output_AML.csv: ['master_cpd_id', 'AML', 'CompoundName', 'GeneSymbolOfTargets', 'TargetOrMechanism']
Размер словаря cpd_name: 545
Примеры из словаря: [(628607, 'foretinib'), (45093, 'N9-isopropylolomoucine'), (687578, 'dinaciclib'), (628613, 'regorafenib'), (660132, 'serdemetan:SCH-529074 (1:1 mol/mol)')]
Заполнено cpd_name в output_ALL.csv: 545
Заполнено cpd_name в output_AML.csv: 545
Обработка завершена. Файлы output_ALL.csv и output_AML.csv обновлены.


In [31]:
import pandas as pd

# Чтение входного CSV-файла
input_file = r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\PortalCompounds.csv"
df = pd.read_csv(input_file)

# Создаём пустой список для хранения новых строк
new_rows = []

# Обработка каждой строки исходного файла
for index, row in df.iterrows():
    # Проверяем, есть ли значение в столбце GeneSymbolOfTargets и не является ли оно пустым
    if pd.notna(row['GeneSymbolOfTargets']) and row['GeneSymbolOfTargets'].strip():
        compound_id = row['CompoundID']
        compound_name = row['CompoundName']
        target_or_mechanism = row.get('TargetOrMechanism', '')
        sample_ids = row.get('SampleIDs', '')
        # Разделяем гены по точке с запятой
        genes = row['GeneSymbolOfTargets'].split(';')
        # Для каждого гена создаём новую строку
        for gene in genes:
            # Удаляем начальные пробелы из названия гена
            cleaned_gene = gene.strip()
            new_rows.append({
                'Gene': cleaned_gene,
                'CompoundID': compound_id,
                'CompoundName': compound_name,
                'TargetOrMechanism': target_or_mechanism,
                'SampleIDs': sample_ids
            })

# Создаём новый DataFrame из списка новых строк
new_df = pd.DataFrame(new_rows, columns=['Gene', 'CompoundID', 'CompoundName', 'TargetOrMechanism', 'SampleIDs'])

# Сортируем по столбцу Gene
new_df = new_df.sort_values(by='Gene')

# Сохраняем результат в новый CSV-файл
output_file = r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\ReformattedCompounds.csv"
new_df.to_csv(output_file, index=False)

print(f"Новый файл сохранён по пути: {output_file}")

Новый файл сохранён по пути: C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\ReformattedCompounds.csv


In [34]:
import pandas as pd

# Чтение файлов
compounds = pd.read_csv(r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\ReformattedCompounds.csv")
id_mapping = pd.read_csv(r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\final_filtered_dedup_merged_result.csv")
z_scores = pd.read_csv(r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\drug_diseases_avg_zscore.csv")

# Проверяем наличие столбцов
print("Столбцы в ReformattedCompounds.csv:", compounds.columns.tolist())
print("Столбцы в final_filtered_dedup_merged_result.csv:", id_mapping.columns.tolist())
print("Столбцы в drug_diseases_avg_zscore.csv:", z_scores.columns.tolist())

# Оставляем только нужные столбцы в z_scores
z_scores = z_scores[['master_cpd_id', 'ALL', 'AML']]

# Функция для извлечения BRD-K... из SampleIDs
def extract_broad_id(sample):
    if not sample or pd.isna(sample):
        return None
    sample = sample.strip()
    if sample.startswith('BRD:'):
        sample = sample[4:]  # Удаляем BRD:
    if not sample.startswith('BRD-K'):
        return None
    parts = sample.split('-')
    if len(parts) >= 2:
        return f"{parts[0]}-{parts[1]}"  # Например, BRD-K62353271
    return None

# Функция для поиска broad_cpd_id в SampleIDs
def find_broad_id(sample_ids, broad_ids):
    if pd.isna(sample_ids):
        return None
    sample_list = str(sample_ids).split(';')
    for sample in sample_list:
        extracted_id = extract_broad_id(sample)
        if extracted_id and extracted_id in broad_ids:
            return extracted_id
    return None

# Уникальные broad_cpd_id из id_mapping
id_mapping = id_mapping.drop_duplicates(subset='broad_cpd_id', keep='first')
broad_id_set = set(id_mapping['broad_cpd_id'].astype(str).str.strip())

# Проверяем пересечение SampleIDs и broad_cpd_id
all_sample_ids = set()
for sample in compounds['SampleIDs'].dropna():
    for s in str(sample).split(';'):
        extracted_id = extract_broad_id(s)
        if extracted_id:
            all_sample_ids.add(extracted_id)

common_ids = all_sample_ids.intersection(broad_id_set)
print(f"Количество общих broad_cpd_id: {len(common_ids)}")
if common_ids:
    print("Примеры общих broad_cpd_id:", list(common_ids)[:5])
else:
    print("Совпадений broad_cpd_id не найдено!")
    print("Примеры обработанных SampleIDs:", list(all_sample_ids)[:5])
    print("Примеры broad_cpd_id:", list(broad_id_set)[:5])

# Создаём копии входного файла для выходных данных
output_all = compounds.copy()
output_aml = compounds.copy()

# Добавляем столбцы ALL и AML
output_all['ALL'] = None
output_aml['AML'] = None

# Добавляем broad_cpd_id в compounds
compounds['found_broad_id'] = compounds['SampleIDs'].apply(
    lambda x: find_broad_id(x, broad_id_set)
)

# Логируем совпадения
matched_count = compounds['found_broad_id'].notna().sum()
print(f"Найдено совпадений broad_cpd_id: {matched_count}")
if matched_count > 0:
    print("Примеры найденных broad_cpd_id:", compounds['found_broad_id'].dropna().head().tolist())

# Фильтруем строки с найденными broad_cpd_id
matched = compounds[compounds['found_broad_id'].notna()]

# Соединяем с id_mapping для получения master_cpd_id
matched = matched.merge(
    id_mapping[['broad_cpd_id', 'master_cpd_id']],
    left_on='found_broad_id',
    right_on='broad_cpd_id',
    how='left'
)

# Логируем master_cpd_id
master_count = matched['master_cpd_id'].notna().sum()
print(f"Найдено master_cpd_id: {master_count}")

# Соединяем с z_scores для получения ALL и AML
matched = matched.merge(
    z_scores[['master_cpd_id', 'ALL', 'AML']],
    on='master_cpd_id',
    how='left'
)

# Обновляем выходные датафреймы
for _, row in matched.iterrows():
    mask = (output_all['SampleIDs'] == row['SampleIDs']) & (output_all['Gene'] == row['Gene'])
    if mask.any():
        output_all.loc[mask, 'ALL'] = row['ALL']
    
    mask = (output_aml['SampleIDs'] == row['SampleIDs']) & (output_aml['Gene'] == row['Gene'])
    if mask.any():
        output_aml.loc[mask, 'AML'] = row['AML']

# Сохраняем результаты
output_all.to_csv(r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\output_ALL.csv", index=False)
output_aml.to_csv(r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\output_AML.csv", index=False)

print("Обработка завершена. Файлы сохранены как output_ALL.csv и output_AML.csv")

Столбцы в ReformattedCompounds.csv: ['Gene', 'CompoundID', 'CompoundName', 'TargetOrMechanism', 'SampleIDs']
Столбцы в final_filtered_dedup_merged_result.csv: ['experiment_id', 'area_under_curve', 'master_cpd_id', 'master_ccl_id', 'cpd_name', 'broad_cpd_id', 'ccl_name', 'ModelID', 'primary_disease_renamed']
Столбцы в drug_diseases_avg_zscore.csv: ['master_cpd_id', 'ALL', 'AML', 'Bile Duct Cancer', 'Bladder Cancer', 'Bone Cancer', 'Breast Cancer', 'CLL', 'CML', 'Cervical Cancer', 'Colon/Colorectal Cancer', 'Endometrial/Uterine Cancer', 'Esophageal Cancer', 'Ewing Sarcoma', 'Gallbladder Cancer', 'Gastric Cancer', 'Glioma', 'Head and Neck Cancer', 'Kidney Cancer', 'Liver Cancer', 'Lung Cancer', 'Lymphoma', 'Medulloblastoma', 'Melanoma', 'Myeloma', 'Neuroblastoma', 'Non-Cancerous', 'Ovarian Cancer', 'Pancreatic Cancer', 'Prostate Cancer', 'Rhabdoid ATRT Cancer', 'Rhabdomyosarcoma', 'Sarcoma', 'Thyroid Cancer']
Количество общих broad_cpd_id: 218
Примеры общих broad_cpd_id: ['BRD-K26603252',

In [35]:
import pandas as pd

# Пути к файлам
all_file = r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\output_ALL.csv"
aml_file = r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\output_AML.csv"

# Функция для обработки файла
def process_file(file_path, column_name):
    # Чтение файла
    df = pd.read_csv(file_path)
    
    # Подсчёт исходного количества строк
    initial_rows = len(df)
    
    # Удаление строк с пустыми значениями в указанном столбце
    df_cleaned = df.dropna(subset=[column_name])
    
    # Подсчёт оставшихся строк
    remaining_rows = len(df_cleaned)
    
    # Сохранение обновлённого файла
    df_cleaned.to_csv(file_path, index=False)
    
    return initial_rows, remaining_rows

# Обработка файла output_ALL.csv
all_initial, all_remaining = process_file(all_file, 'ALL')
print(f"Файл output_ALL.csv:")
print(f"  Строк изначально: {all_initial}")
print(f"  Строк осталось: {all_remaining}")
print(f"  Удалено строк: {all_initial - all_remaining}")

# Обработка файла output_AML.csv
aml_initial, aml_remaining = process_file(aml_file, 'AML')
print(f"Файл output_AML.csv:")
print(f"  Строк изначально: {aml_initial}")
print(f"  Строк осталось: {aml_remaining}")
print(f"  Удалено строк: {aml_initial - aml_remaining}")

print("Обработка завершена. Файлы обновлены.")

Файл output_ALL.csv:
  Строк изначально: 12242
  Строк осталось: 610
  Удалено строк: 11632
Файл output_AML.csv:
  Строк изначально: 12242
  Строк осталось: 610
  Удалено строк: 11632
Обработка завершена. Файлы обновлены.


In [46]:
import pandas as pd
import os

# Пути к входным файлам
all_file = r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\ALL.csv"
aml_file = r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\AML.csv"
depscore_all_file = r"C:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\r_visualizations\scatter_plots\mean_gene_z-score_depscore_by_cancer_type\mean_gene_z-score_depscore_data\mean_gene_z-score_depscore_all.csv"
depscore_aml_file = r"C:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\r_visualizations\scatter_plots\mean_gene_z-score_depscore_by_cancer_type\mean_gene_z-score_depscore_data\mean_gene_z-score_depscore_aml.csv"

# Создаем папку для результатов, если её нет
output_dir = r"C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\results"
os.makedirs(output_dir, exist_ok=True)

# Читаем CSV файлы
df_all = pd.read_csv(all_file)
df_aml = pd.read_csv(aml_file)
df_depscore_all = pd.read_csv(depscore_all_file)
df_depscore_aml = pd.read_csv(depscore_aml_file)

# Группируем по генам и вычисляем среднее для ALL
all_means = df_all.groupby('Gene')['ALL'].mean().reset_index()
all_means.columns = ['Gene', 'Mean_Z_Score']

# Группируем по генам и вычисляем среднее для AML
aml_means = df_aml.groupby('Gene')['AML'].mean().reset_index()
aml_means.columns = ['Gene', 'Mean_Z_Score']

# Объединяем с данными depscore (left join, чтобы сохранить все гены)
all_means = all_means.merge(df_depscore_all[['Gene', 'Mean_Zscore']], on='Gene', how='left')
all_means = all_means.rename(columns={'Mean_Zscore': 'Mean_Zscore (depscore)'})

aml_means = aml_means.merge(df_depscore_aml[['Gene', 'Mean_Zscore']], on='Gene', how='left')
aml_means = aml_means.rename(columns={'Mean_Zscore': 'Mean_Zscore (depscore)'})

# Подсчитываем совпадения и пустые строки
all_matches = all_means['Mean_Zscore (depscore)'].notna().sum()
all_empty = all_means['Mean_Zscore (depscore)'].isna().sum()
aml_matches = aml_means['Mean_Zscore (depscore)'].notna().sum()
aml_empty = aml_means['Mean_Zscore (depscore)'].isna().sum()

# Сохраняем результаты в CSV файлы
all_means.to_csv(os.path.join(output_dir, 'gene_mean_z_scores_all_with_depscore.csv'), index=False)
aml_means.to_csv(os.path.join(output_dir, 'gene_mean_z_scores_aml_with_depscore.csv'), index=False)

# Выводим информацию о совпадениях и пустых строках
print("Результаты для ALL:")
print(f"Совпадений (генов с Mean_Zscore (depscore)): {all_matches}")
print(f"Пустых строк (генов без Mean_Zscore (depscore)): {all_empty}")
print(f"Файл сохранён: {os.path.join(output_dir, 'gene_mean_z_scores_all_with_depscore.csv')}")
print("\nРезультаты для AML:")
print(f"Совпадений (генов с Mean_Zscore (depscore)): {aml_matches}")
print(f"Пустых строк (генов без Mean_Zscore (depscore)): {aml_empty}")
print(f"Файл сохранён: {os.path.join(output_dir, 'gene_mean_z_scores_aml_with_depscore.csv')}")

Результаты для ALL:
Совпадений (генов с Mean_Zscore (depscore)): 299
Пустых строк (генов без Mean_Zscore (depscore)): 0
Файл сохранён: C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\results\gene_mean_z_scores_all_with_depscore.csv

Результаты для AML:
Совпадений (генов с Mean_Zscore (depscore)): 299
Пустых строк (генов без Mean_Zscore (depscore)): 0
Файл сохранён: C:\Users\rusla\OneDrive\Рабочий стол\Анализ препаратов\ctd2\results\gene_mean_z_scores_aml_with_depscore.csv
